In [1]:
import pandas as pd
import sklearn
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn import model_selection, naive_bayes, svm, metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score


In [2]:
df = pd.read_csv('reports_with_vectors_1000.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112095 entries, 0 to 112094
Columns: 1009 entries, Unnamed: 0 to you
dtypes: float64(1005), int64(1), object(3)
memory usage: 862.9+ MB


In [4]:
df['region'].value_counts()

southeast             23140
pacific               21094
midwest               19920
northeast             16152
southwest             10848
other                  8660
rockies                6533
ontario                2027
noncontiguous          1024
british_columbia       1005
prarie_provinces        883
atlantic_provinces      413
quebec                  345
north_canada             50
Name: region, dtype: int64

In [5]:
df = df[pd.notnull(df['region'])]

In [6]:
# one-hot encoded shape feature
cat_cols = ['shape']

for col in cat_cols:
    temp = pd.get_dummies(df[col], prefix=col)
    df= pd.concat([df, temp], axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y= Encoder.fit_transform(df['region'])

In [8]:
d = dict(zip(df['region'], y))
d = pd.DataFrame(
sorted(d.items(), key=lambda x: x[1], reverse=False))

In [9]:
d

,0,1
0,atlantic_provinces,0
1,british_columbia,1
2,midwest,2
3,noncontiguous,3
4,north_canada,4
5,northeast,5
6,ontario,6
7,other,7
8,pacific,8
9,prarie_provinces,9


In [10]:
x = df.drop(columns =['shape', 'Unnamed: 0','citystate','region','city_longitude','city_latitude'])

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=1,stratify=y)

In [12]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=1, stratify=y_train)


In [13]:
lgb_mdl_rand = lgb.LGBMClassifier(boosting_type = 'gbdt',
                                n_jobs = -1,
                                objective = 'multiclass',
                                num_iterations = 3,
                                metric = 'multi_logloss',
                                pos_bagging_fraction = 0.5,
                                  
                               )

lgb_grid_params_rand = { 
    'learning_rate': [0.5, 1, 3, 5],
    'estimator__max_depth' : [10, 20, 30],
    'num_leaves': [5, 10,15,20],
    
}

lgb_gs_rand = RandomizedSearchCV(lgb_mdl_rand, lgb_grid_params_rand,
                                 n_jobs=1, cv=4,
                                 random_state=2)

fit_params={"early_stopping_rounds" : 10,
           "eval_set" : [[x_val, y_val]]}


In [14]:
lgb_gs_rand.fit(x_train, y_train, **fit_params)

C:\Users\Lucyc\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 2.36421
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 2.20192
[3]	valid_0's multi_logloss: 2.59947
Did not meet early stopping. Best iteration is:
[2]	valid_0's multi_logloss: 2.20192
[1]	valid_0's multi_logloss: 2.38961
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 2.25034
[3]	valid_0's multi_logloss: 2.717
Did not meet early stopping. Best iteration is:
[2]	valid_0's multi_logloss: 2.25034
[1]	valid_0's multi_logloss: 2.37314
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 2.28007
[3]	valid_0's multi_logloss: 2.76593
Did not meet early stopping. Best iteration is:
[2]	valid_0's multi_logloss: 2.28007
[1]	valid_0's multi_logloss: 2.37394
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 2.23254
[3]	valid_0's multi_logloss: 2.68277
Did not meet early stopping. Best iteration is:
[2]	valid_0

[1]	valid_0's multi_logloss: 3.17819
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 5.13633
[3]	valid_0's multi_logloss: 12.2939
Did not meet early stopping. Best iteration is:
[1]	valid_0's multi_logloss: 3.17819
[1]	valid_0's multi_logloss: 3.32313
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 5.19261
[3]	valid_0's multi_logloss: 5.82244
Did not meet early stopping. Best iteration is:
[1]	valid_0's multi_logloss: 3.32313
[1]	valid_0's multi_logloss: 3.3709
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 6.15082
[3]	valid_0's multi_logloss: 6.24948
Did not meet early stopping. Best iteration is:
[1]	valid_0's multi_logloss: 3.3709
[1]	valid_0's multi_logloss: 3.29832
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 5.32811
[3]	valid_0's multi_logloss: 6.54787
Did not meet early stopping. Best iteration is:
[1]	valid_0

RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            metric='multi_logloss',
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_iterations=3, num_leaves=31,
                                            objective=...
                                            random_state=None, reg_alpha=0.0,
                                            reg_l

In [15]:
lgb_train_predictions =lgb_gs_rand.predict(x_train)
lgb_test_predictions= lgb_gs_rand.predict(x_test)

train_probs_lgb_r = lgb_gs_rand.predict_proba(x_train)
test_probs_lgb_r = lgb_gs_rand.predict_proba(x_test)


In [16]:
accuracy_score(lgb_train_predictions, y_train)

0.29528854195706716

In [17]:
accuracy_score(lgb_test_predictions, y_test)

0.2825282126767474

In [18]:
from sklearn.metrics import log_loss
log_loss(y_train, train_probs_lgb_r)

2.188777229010912

In [19]:
log_loss(y_test, test_probs_lgb_r)

2.2882975532677867

In [20]:
pd.crosstab(y_test, lgb_test_predictions,
            rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,3,4,5,6,7,8,9,10,11,12,13,All
True,,,,,,,,,,,,,,,
0,2,1,14,0,1,4,1,5,7,1,1,0,45,0,82
1,1,31,24,3,4,6,4,11,23,7,0,4,82,1,201
2,28,14,1004,4,22,113,39,102,317,16,24,5,2279,17,3984
3,2,1,20,4,0,5,0,5,20,1,2,3,141,1,205
4,0,0,4,0,0,0,0,0,2,0,0,0,4,0,10
5,22,7,509,12,12,265,23,117,215,8,21,13,1996,10,3230
6,3,3,110,1,0,8,13,21,36,4,0,0,206,0,405
7,14,16,215,14,14,64,13,212,205,7,19,3,927,9,1732
8,19,16,557,31,28,59,17,124,1345,10,20,38,1935,20,4219


In [21]:
lgb_gs_rand.best_params_

{'num_leaves': 20, 'learning_rate': 1, 'estimator__max_depth': 20}

In [25]:
import shap

lgb_clf = lgb.LGBMClassifier(boosting_type = 'gbdt',
                                n_jobs = -1,
                                objective = 'multiclass',
                                num_iterations = 3,
                                metric = 'multi_logloss',
                                pos_bagging_fraction = 0.5,
                    num_leaves= 20, learning_rate= 1, estimator__max_depth=20)

lgb_clf.fit(x_train,y_train)
                
region_shap_explainer = shap.TreeExplainer(lgb_clf)

In [26]:
region_shap_vals_train = region_shap_explainer.shap_values(x_train)

In [27]:
region_shap_vals_test = region_shap_explainer.shap_values(x_test)

In [28]:
# We can also plot the variable importance of all of our variables in a summary plot
shap.summary_plot(region_shap_vals_train, x_train)

MemoryError: 